# Buy/Not buy action predicted using LSTM

This notebook utilised LSTM to predict the buy or not buy action of a stock. The feature of the stock is the Open, High, Low, Close, Volume, Adj Close price of the stock. The target is the action of the stock, which is 1 if the stock price is higher than the previous day, and 0 if the stock price is lower than the previous day.

In [1]:
# import library for Support Vector Classification
from sklearn.svm import SVC

# import library for plot
import matplotlib.pyplot as plt

# import library for data handling
import pandas_datareader as web
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import math

# import library for pytorch
import torch
import torch.nn as nn

# import custom function
from stock_predict import SequenceDataset, ShallowRegressionLSTM, train, test, get_stock_data, split_train_test, predict, split_train_test_binary

# Get cuda device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

/home/victoru/anaconda3/envs/testpython/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Get the past stock price
df = get_stock_data(stock_name = 'TSLA', source = 'yahoo', past_year = 3)

In [3]:
features = df.columns

In [4]:
features

Index(['High', 'Low', 'Open', 'Close', 'Volume', 'Adj Close'], dtype='object')

In [ ]:
# set up train and test data
train_df, test_df, train_size, test_size, df['perfect'] = split_train_test_binary(df,0.7)


In [ ]:
# create dataset and dataloader
batch_size = 4
seq_len = 60

torch.manual_seed(seq_len*batch_size/2)

train_dataset = SequenceDataset(train_df, 'perfect', features, seq_len)
test_dataset = SequenceDataset(test_df, 'perfect', features, seq_len)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# get the first batch of data to check the shape
x, y = next(iter(train_loader))
print("Feature shape: " + x.shape)
print("Target shape: " + y.shape)

In [ ]:
# create model, loss and optimizer
learning_rate = 5e-5
num_hidden = 16
num_layers = 2
epoch = 40

model = ShallowRegressionLSTM(n_features = len(features), n_hidden = num_hidden, n_layers = num_layers, device = device).to(device)
loss = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)